# Introduction

**SUMMARY**

The purpose of this report is to analyze health inequities in the US by county and how those inequities influence the degree of impact of the COVID-19 pandemic.

____

**STAKEHOLDERS**

Federal, state, and local governments and epidemiologists can use this analysis to shape public health policy and pandemic response plans. The goal is to learn from COVID-19 to prevent the loss that occurred in 2020-2021 from happening again.

____

**DATA SOURCES**

- [CDC’s Social Vulnerability Index](https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2018.html)

- [County Health Rankings](https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources) 

- [New York Times COVID cases and deaths](https://github.com/nytimes/covid-19-data)

- [CDC'S COVID-19 Vaccinations](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh)

- [CDC's Vaccine Hesitancy](https://data.cdc.gov/Vaccinations/Vaccine-Hesitancy-for-COVID-19-County-and-local-es/q9mh-h2tw)

____

**METHODS**

Classification model which predicts a given county’s degree of impact of the pandemic (high impact, average impact, low impact). Feature importance and coefficients will be analyzed to recommend what features put communities at greater risk in order to design policies to address the underlying issues.


In [1]:
# style notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [2]:
#import standard packages
import pandas as pd
pd.set_option('display.max_columns', 0)
import numpy as np

#import viz packages
import matplotlib.pyplot as plt
%matplotlib inline
#plt.rcParams['figure.figsize'] = (15,5) #setting figures to timeseries-friendly
import plotly.express as px
import plotly.graph_objects as go

#import time series tools from statsmodels
import statsmodels.tsa.api as tsa
import statsmodels
print(f'Statsmodels version = {statsmodels.__version__}')

Statsmodels version = 0.12.0


# Data Load

Data files are too large to push to github in their raw form, so the first step is to compress the files.

In [3]:
nyt_cases_deaths = pd.read_csv('data/NYT-covid-cases-deaths.csv')
nyt_cases_deaths

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1462594,2021-06-27,Sweetwater,Wyoming,56037.0,4729,40.0
1462595,2021-06-27,Teton,Wyoming,56039.0,3801,11.0
1462596,2021-06-27,Uinta,Wyoming,56041.0,2315,13.0
1462597,2021-06-27,Washakie,Wyoming,56043.0,925,26.0


In [4]:
nyt_cases_deaths.to_csv('data_compressed/NYT-covid-cases-deaths.csv.gz', index=False, compression='gzip')

In [5]:
nyt_cases_deaths = pd.read_csv('data_compressed/NYT-covid-cases-deaths.csv.gz', compression='gzip')
nyt_cases_deaths

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1462594,2021-06-27,Sweetwater,Wyoming,56037.0,4729,40.0
1462595,2021-06-27,Teton,Wyoming,56039.0,3801,11.0
1462596,2021-06-27,Uinta,Wyoming,56041.0,2315,13.0
1462597,2021-06-27,Washakie,Wyoming,56043.0,925,26.0


In [6]:
svi = pd.read_csv('data/SVI2018_US_COUNTY.csv')
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,...,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,...,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,...,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,...,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,...,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,...,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8564,0.8077,2.4628,0.8303,0.6616,0.5791,1.2407,0.6507,0.5931,0.6396,0.4056,0.5142,0.6250,2.7775,0.6492,9.4046,0.8252,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10686,796,9.4,0.7,117894


In [7]:
svi.to_csv('data_compressed/SVI2018_US_COUNTY.csv.gz', compression='gzip', index=False)

In [8]:
svi = pd.read_csv('data_compressed/SVI2018_US_COUNTY.csv.gz', compression='gzip')
svi.head()

,ST,STATE,ST_ABBR,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,...,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP
0,35,NEW MEXICO,NM,Rio Arriba,35039,"Rio Arriba County, New Mexico",5860.869195,39307,0,20044,71,12398,439,-999,-999,-999,-999,-999,-999,3669,426,7083,25,9318,14,6280,495,1330,285.0,34397,145.0,755,209.5,67,37.1,7770,431,264,77.1,763,160,654,142,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.8,1.6,18.0,0.1,23.7,0.0,16.1,...,2.1,0.6,6.2,1.3,1.7,0.4,-999.0000,-999.0000,-999.0000,0.5922,-999.0000,-999.0000,0.4893,0.6826,0.5610,0.8383,2.5712,0.8758,0.9917,0.7740,1.7657,0.9268,0.0551,0.9869,0.5498,0.5788,0.4126,2.5832,0.5409,-999.0000,-999.0000,-999,-999,-999,0,-999,0,0,0,0,0,1,0,1,0,1,0,0,0,1,-999,4160,588,10.6,1.5,32290
1,1,ALABAMA,AL,Autauga,1001,"Autauga County, Alabama",594.443459,55200,0,23315,71,21115,383,8422,1137,1065,257,29372,2306,4204,475,8050,75,13369,32,10465,729,1586,319.9,13788,59.0,426,205.9,886,308.7,4279,469,299,142.3,1191,272,546,161,15.4,2.1,4.2,1.0,29372.0,2306.0,11.3,1.3,14.6,0.1,24.2,0.1,19.3,...,1.4,0.7,5.6,1.3,1.0,0.3,0.5401,0.2745,0.2860,0.4397,1.5403,0.3631,0.1850,0.7529,0.7905,0.3792,2.1076,0.5810,0.6336,0.5113,1.1449,0.5947,0.6017,0.7408,0.2964,0.4846,0.1525,2.2760,0.3741,7.0688,0.4354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3875,508,7.1,0.9,37301
2,1,ALABAMA,AL,Blount,1009,"Blount County, Alabama",644.830460,57645,0,24222,55,20600,396,8220,992,909,193,22656,905,7861,727,10233,91,13468,53,8114,592,1437,267.2,7413,229.0,934,239.3,211,104.2,6108,476,339,130.7,856,201,543,117,14.4,1.7,4.1,0.8,22656.0,905.0,19.8,1.8,17.8,0.2,23.4,0.1,14.2,...,1.6,0.6,4.2,1.0,0.9,0.2,0.4723,0.2611,0.7561,0.8405,2.3300,0.6143,0.4715,0.6406,0.3763,0.2961,1.7845,0.3187,0.4206,0.7170,1.1376,0.5915,0.1512,0.8816,0.3703,0.2420,0.1165,1.7616,0.1741,7.0137,0.4242,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6303,732,11.0,1.3,40036
3,1,ALABAMA,AL,Butler,1013,"Butler County, Alabama",776.838201,20025,0,10026,51,6708,274,4640,521,567,147,20430,1258,2141,268,3806,21,4566,88,3492,370,704,143.9,9641,22.0,93,137.4,134,47.4,2625,212,119,57.7,520,102,322,88,23.5,2.6,6.7,1.7,20430.0,1258.0,15.4,1.9,19.0,0.1,22.8,0.4,17.7,...,1.8,0.9,7.8,1.5,1.6,0.4,0.8860,0.6968,0.8879,0.6753,3.1460,0.8455,0.5928,0.5578,0.6845,0.8185,2.6536,0.9077,0.8711,0.3582,1.2293,0.6447,0.2416,0.8940,0.4457,0.7685,0.3792,2.7290,0.6259,9.7579,0.8653,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2005,340,10.2,1.7,17280
4,1,ALABAMA,AL,Calhoun,1015,"Calhoun County, Alabama",605.867251,115098,0,53682,184,45033,683,20819,1317,4628,526,24706,758,12620,766,19386,119,25196,66,23598,1086,4701,464.0,31675,34.0,1076,250.2,1990,303.0,7904,546,772,206.2,2599,331,3112,436,18.6,1.2,8.8,1.0,24706.0,758.0,15.9,1.0,16.8,0.1,21.9,0.1,20.8,...,1.7,0.5,5.8,0.7,2.7,0.4,0.7322,0.8850,0.6076,0.6988,2.9236,0.7866,0.3664,0.4323,0.8564,0.8077,2.4628,0.8303,0.6616,0.5791,1.2407,0.6507,0.5931,0.6396,0.4056,0.5142,0.6250,2.7775,0.6492,9.4046,0.8252,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10686,796,9.4,0.7,117894


In [9]:
county_health_rankings = pd.read_csv('data/countyhealthrankings-analytic_data2021.csv', header=[0,1])
county_health_rankings.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed),Premature death (AIAN),Premature death CI low (AIAN),Premature death CI high (AIAN),Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Asian/Pacific Islander),Premature death CI low (Asian/Pacific Islander),Premature death CI high (Asian/Pacific Islander),Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Black),Premature death CI low (Black),Premature death CI high (Black),Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Hispanic),Premature death CI low (Hispanic),Premature death CI high (Hispanic),Premature death flag (Hispanic) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (White),Premature death CI low (White),Premature death CI high (White),Premature death flag (White) (. = No Flag/1=Unreliable/2=Suppressed),Poor or fair health raw value,Poor or fair health numerator,Poor or fair health denominator,Poor or fair health CI low,Poor or fair health CI high,Poor physical health days raw value,Poor physical health days numerator,Poor physical health days denominator,Poor physical health days CI low,Poor physical health days CI high,Poor mental health days raw value,Poor mental health days numerator,Poor mental health days denominator,Poor mental health days CI low,Poor mental health days CI high,Low birthweight raw value,Low birthweight numerator,Low birthweight denominator,Low birthweight CI low,Low birthweight CI high,LBW unreliable indicator (Unreliable = Numerator < 20 or relative standard error > 20%),Low birthweight (AIAN),Low birthweight CI low (AIAN),...,Population CI high,% below 18 years of age raw value,% below 18 years of age numerator,% below 18 years of age denominator,% below 18 years of age CI low,% below 18 years of age CI high,% 65 and older raw value,% 65 and older numerator,% 65 and older denominator,% 65 and older CI low,% 65 and older CI high,% Non-Hispanic Black raw value,% Non-Hispanic Black numerator,% Non-Hispanic Black denominator,% Non-Hispanic Black CI low,% Non-Hispanic Black CI high,% American Indian & Alaska Native raw value,% American Indian & Alaska Native numerator,% American Indian & Alaska Native denominator,% American Indian & Alaska Native CI low,% American Indian & Alaska Native CI high,% Asian raw value,% Asian numerator,% Asian denominator,% Asian CI low,% Asian CI high,% Native Hawaiian/Other Pacific Islander raw value,% Native Hawaiian/Other Pacific Islander numerator,% Native Hawaiian/Other Pacific Islander denominator,% Native Hawaiian/Other Pacific Islander CI low,% Native Hawaiian/Other Pacific Islander CI high,% Hispanic raw value,% Hispanic numerator,% Hispanic denominator,% Hispanic CI low,% Hispanic CI high,% Non-Hispanic White raw value,% Non-Hispanic White numerator,% Non-Hispanic White denominator,% Non-Hispanic White CI low,% Non-Hispanic White CI high,% not proficient in English raw value,% not proficient in English numerator,% not proficient in English denominator,% not proficient in English CI low,% not proficient in English CI high,% Females raw value,% Females numerator,% Females denominator,% Females CI low,% Females CI high,% Rural raw value,% Rural numerator,% Rural denominator,% Rural CI low,% Rural CI high
,statecode,countycode,fipscode,state,county,year,county_ranked,v001_rawvalue,v001_numerator,v001_denominator,v001_cilow,v001_cihigh,v001_flag,v001_race_aian,v001_race_aian_cilow,v001_race_aian_cihigh,v001_race_aian_flag,v001_race_asian,v001_race_asian_cilow,v001_race_asian_cihigh,v001_race_asian_flag,v001_race_black,v001_race_black_cilow,v001_race_black_cihigh,v001_race_black_flag,v001_race_hispanic,v001_race_hispanic_cilow,v001_rac

In [10]:
county_health_rankings.to_csv('data_compressed/countyhealthrankings-analytic_data2021.csv.gz', compression='gzip',
                             index=False)

In [11]:
county_health_rankings = pd.read_csv('data_compressed/countyhealthrankings-analytic_data2021.csv.gz', 
                                     compression='gzip', header=[0,1])
county_health_rankings.head()

,State FIPS Code,County FIPS Code,5-digit FIPS Code,State Abbreviation,Name,Release Year,County Ranked (Yes=1/No=0),Premature death raw value,Premature death numerator,Premature death denominator,Premature death CI low,Premature death CI high,Premature death flag (0 = No Flag/1=Unreliable/2=Suppressed),Premature death (AIAN),Premature death CI low (AIAN),Premature death CI high (AIAN),Premature death flag (AIAN) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Asian/Pacific Islander),Premature death CI low (Asian/Pacific Islander),Premature death CI high (Asian/Pacific Islander),Premature death flag (Asian/Pacific Islander) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Black),Premature death CI low (Black),Premature death CI high (Black),Premature death flag (Black) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (Hispanic),Premature death CI low (Hispanic),Premature death CI high (Hispanic),Premature death flag (Hispanic) (. = No Flag/1=Unreliable/2=Suppressed),Premature death (White),Premature death CI low (White),Premature death CI high (White),Premature death flag (White) (. = No Flag/1=Unreliable/2=Suppressed),Poor or fair health raw value,Poor or fair health numerator,Poor or fair health denominator,Poor or fair health CI low,Poor or fair health CI high,Poor physical health days raw value,Poor physical health days numerator,Poor physical health days denominator,Poor physical health days CI low,Poor physical health days CI high,Poor mental health days raw value,Poor mental health days numerator,Poor mental health days denominator,Poor mental health days CI low,Poor mental health days CI high,Low birthweight raw value,Low birthweight numerator,Low birthweight denominator,Low birthweight CI low,Low birthweight CI high,LBW unreliable indicator (Unreliable = Numerator < 20 or relative standard error > 20%),Low birthweight (AIAN),Low birthweight CI low (AIAN),...,Population CI high,% below 18 years of age raw value,% below 18 years of age numerator,% below 18 years of age denominator,% below 18 years of age CI low,% below 18 years of age CI high,% 65 and older raw value,% 65 and older numerator,% 65 and older denominator,% 65 and older CI low,% 65 and older CI high,% Non-Hispanic Black raw value,% Non-Hispanic Black numerator,% Non-Hispanic Black denominator,% Non-Hispanic Black CI low,% Non-Hispanic Black CI high,% American Indian & Alaska Native raw value,% American Indian & Alaska Native numerator,% American Indian & Alaska Native denominator,% American Indian & Alaska Native CI low,% American Indian & Alaska Native CI high,% Asian raw value,% Asian numerator,% Asian denominator,% Asian CI low,% Asian CI high,% Native Hawaiian/Other Pacific Islander raw value,% Native Hawaiian/Other Pacific Islander numerator,% Native Hawaiian/Other Pacific Islander denominator,% Native Hawaiian/Other Pacific Islander CI low,% Native Hawaiian/Other Pacific Islander CI high,% Hispanic raw value,% Hispanic numerator,% Hispanic denominator,% Hispanic CI low,% Hispanic CI high,% Non-Hispanic White raw value,% Non-Hispanic White numerator,% Non-Hispanic White denominator,% Non-Hispanic White CI low,% Non-Hispanic White CI high,% not proficient in English raw value,% not proficient in English numerator,% not proficient in English denominator,% not proficient in English CI low,% not proficient in English CI high,% Females raw value,% Females numerator,% Females denominator,% Females CI low,% Females CI high,% Rural raw value,% Rural numerator,% Rural denominator,% Rural CI low,% Rural CI high
,statecode,countycode,fipscode,state,county,year,county_ranked,v001_rawvalue,v001_numerator,v001_denominator,v001_cilow,v001_cihigh,v001_flag,v001_race_aian,v001_race_aian_cilow,v001_race_aian_cihigh,v001_race_aian_flag,v001_race_asian,v001_race_asian_cilow,v001_race_asian_cihigh,v001_race_asian_flag,v001_race_black,v001_race_black_cilow,v001_race_black_cihigh,v001_race_black_flag,v001_race_hispanic,v001_race_hispanic_cilow,v001_rac

# Appendix